# 🌌 N-Body Simulation & Halo Formation

#### Pragun Nepal  
#### June 21, 2025

---

This notebook contains the code for running the N-body Simulation & Halo Formation (with and without axes). The full video of the simulations have been uploaded in my YouTube channel - https://www.youtube.com/@PragunNepal

All the data for this simulation have been obatined from the coupled N-body, FoF-Halo-Finder, and ReionYuga Repositories by Dr. Rajesh Mondal (GitHub : rajeshmondal18 - https://github.com/rajeshmondal18 )

---

## 📚 Table of Contents

1. [Setting up ffmpeg for Jupyter Lab](#Setting_up_ffmpeg_for_Jupyter_Lab)
2. [Code for N-body Simulation & Halos Formation](#Code_for_N-body_Simulation)
3. [Code for N-body Simulation & Halos Formation (with axes)](#Code_for_N-body_Simulation_axes)
4. [Conclusion](#conclusion)

---

## 🔭 <a name="Setting_up_ffmpeg_for_Jupyter_Lab"></a>1. Setting up ffmpeg for Jupyter Lab



```python
import os
import matplotlib as mpl

#Enter the Full Fath of the ffmeg library - It may be different for you
ffmpeg_path = "/opt/homebrew/bin/ffmpeg"

#Prepend its directory to PATH for this session
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ["PATH"]

#Tell Matplotlib exactly where to find it
mpl.rcParams["animation.ffmpeg_path"] = ffmpeg_path

#Verification
import shutil
print("→ PATH is now:", os.environ["PATH"].split(os.pathsep)[:3], "…")
print("→ which(ffmpeg):", shutil.which("ffmpeg"))
print("→ rcParams ffmpeg_path:", mpl.rcParams["animation.ffmpeg_path"])
```



If ffmpeg is not installed in your PC. You can do so by visiting the website - https://ffmpeg.org/download.html



---

## ⚙️ <a name="Code_for_N-body_Simulation"></a>2. Code for N-body Simulation & Halo Formation (without axes)

```python
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.spatial import cKDTree

#Extract Redshift Value
def extract_redshift1(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1] + "." + parts[2])

def extract_z_nbody(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1] + "." + parts[2])

def extract_z_halo(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1])

#Load (N-body only)
files1 = sorted(glob.glob("p_*.txt"), key=extract_redshift1, reverse=True)
z1 = [extract_redshift1(f) for f in files1]
raw1 = [np.loadtxt(f).reshape(-1, 3) for f in files1]

frames1, zs1 = [], []
for (a, za), (b, zb) in zip(zip(raw1[:-1], z1[:-1]), zip(raw1[1:], z1[1:])):
    frames1.append(a); zs1.append(za)
    frames1.append((a + b) / 2); zs1.append((za + zb) / 2)
frames1.append(raw1[-1]); zs1.append(z1[-1])

#Load (N-body for center panel)
#Reusing N-body part

frames2_n = frames1.copy()

#I have assumed that the N-body Data Files are in the same directory as this notebook
#The Halo Files are in a different directory

#If n-body data files are also in a different directory, the same approach as below can be taken for n-body as well

#Load (halo-only) from external path
halo_dir = "/Users/pragunnepal/Downloads/500Halos" #Path for the Files
halo_files = sorted(glob.glob(os.path.join(halo_dir, "halo_*.txt")), key=extract_z_halo, reverse=True)
z_h = [extract_z_halo(f) for f in halo_files]
raw_h = [np.loadtxt(f).reshape(-1, 4) for f in halo_files]

#Interpolate halo data for right and center (same frames)

frames3 = []
for z in zs1:
    if z in z_h:
        d3 = raw_h[z_h.index(z)]
    else:
        all_z = np.array(z_h)
        if len(all_z) == 0:
            d3 = np.zeros((0, 4))
        elif z <= all_z[-1]:
            d3 = raw_h[-1]
        elif z >= all_z[0]:
            d3 = raw_h[0]
        else:
            i = np.searchsorted(-all_z, -z)
            i = np.clip(i, 1, len(all_z) - 1)
            zA, zB = z_h[i - 1], z_h[i]
            A, B = raw_h[i - 1], raw_h[i]
            t = (z - zA) / (zB - zA) if zB != zA else 0
            if A.size == 0 or B.size == 0:
                d3 = A if B.size == 0 else B
            else:
                tree = cKDTree(A[:, 1:])
                _, idxs = tree.query(B[:, 1:], k=1)
                A_matched = A[idxs]
                d3 = (1 - t) * A_matched + t * B
    frames3.append(d3)

#Assign halo in center panel same as right panel

frames2 = list(zip(frames2_n, frames3, zs1))

#Animation setup

fps = 30
extra_hold = fps * 33
zoom_in, zoom_out, max_zoom = fps * 15, fps * 5, 1.2
motion = len(frames1)
total_frames = motion + extra_hold

samples = np.vstack([fr[:, :3] for fr in frames1] + [fr[0] for fr in frames2] + [fr[:, 1:] for fr in frames3 if fr.size > 0])
bounds = [(samples[:, i].min(), samples[:, i].max()) for i in range(3)]
bounds = [(mn - 0.2 * (mx - mn), mx + 0.2 * (mx - mn)) for mn, mx in bounds]
centers = [0.5 * (mn + mx) for mn, mx in bounds]

dpi = 300
fig = plt.figure(figsize=(18, 6), dpi=dpi)
axes = [fig.add_subplot(1, 3, i + 1, projection='3d') for i in range(3)]
for ax in axes:
    ax.set_facecolor('black')
    fig.patch.set_facecolor('black')
    ax.axis('off')
    ax.set_box_aspect([1, 1, 1])
txt = axes[0].text2D(0.02, 0.95, '', transform=axes[0].transAxes, color='white')
ax_triad = inset_axes(axes[0], width="20%", height="20%", loc='lower left')
ax_triad.axis('off')

scats1 = axes[0].scatter([], [], [], s=0.5, alpha=0.1, c='cyan', edgecolors='none')
scats2_n = axes[1].scatter([], [], [], s=0.5, alpha=0.1, c='cyan', edgecolors='none')
scats2_h = axes[1].scatter([], [], [], s=[], alpha=0.7, c='gold', edgecolors='none')
scats3 = axes[2].scatter([], [], [], s=[], c='gold', alpha=0.7, edgecolors='none')

def rotation_matrix(angle_deg):
    theta = np.deg2rad(angle_deg)
    Rz = np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
    Rx = np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
    return Rz @ Rx

def update(frame):
    idx = frame if frame < motion else motion - 1
    d1 = frames1[idx]
    scats1._offsets3d = (d1[:, 0], d1[:, 1], d1[:, 2])
    dn, dh, _ = frames2[idx]
    scats2_n._offsets3d = (dn[:, 0], dn[:, 1], dn[:, 2])
    if dh.size:
        scats2_h._offsets3d = (dh[:, 1], dh[:, 2], dh[:, 3])
        scats2_h.set_sizes(np.cbrt(np.ones(len(dh))) * 30)
    else:
        scats2_h._offsets3d = ([], [], [])
        scats2_h.set_sizes([])
    d3 = frames3[idx]
    if d3.size:
        coords3 = d3[:, 1:]
        sizes3 = (np.cbrt(d3[:, 0]) / np.cbrt(d3[:, 0]).max()) * 50
        scats3._offsets3d = (coords3[:, 0], coords3[:, 1], coords3[:, 2])
        scats3.set_sizes(sizes3)
    else:
        scats3._offsets3d = ([], [], [])
        scats3.set_sizes([])
    txt.set_text(f'z = {zs1[idx]:.3f}')
    angle = frame * 0.1
    for ax in axes:
        ax.view_init(25 + angle, 30 + angle)
    ax_triad.clear()
    ax_triad.axis('off')
    R = rotation_matrix(angle)
    for color, vec, label in zip(['red', 'green', 'blue'], [np.array([1, 0, 0]), np.array([0, 1, 0]), np.array([0, 0, 1])], ['X', 'Y', 'Z']):
        vrot = R @ vec
        end = np.array([0.5, 0.5]) + 0.3 * vrot[:2]
        ax_triad.annotate('', xy=end, xytext=(0.5, 0.5), arrowprops=dict(arrowstyle='->', color=color))
        ax_triad.text(end[0], end[1], label, color=color, fontsize=8)
    if frame >= motion:
        h = frame - motion
        if h <= zoom_in:
            f = 1 + (max_zoom - 1) * (h / zoom_in)
        else:
            f = max_zoom - (max_zoom - 1) * min((h - zoom_in) / zoom_out, 1)
    else:
        f = 1
    for i, (mn, mx) in enumerate(bounds):
        c = (mn + mx) / 2
        r = (mx - mn) / 2
        low, high = c - r / f, c + r / f
        for ax in axes:
            if i == 0:
                ax.set_xlim(low, high)
            elif i == 1:
                ax.set_ylim(low, high)
            else:
                ax.set_zlim(low, high)
    return scats1, scats2_n, scats2_h, scats3, txt

ani = FuncAnimation(fig, update, frames=total_frames, interval=1000 / fps, blit=False)
writer = FFMpegWriter(fps=fps, metadata={'PragunNepal': 'Combined'}, bitrate=30000)
ani.save('N-body_Simulation_&_Halo_Formation.mp4', writer=writer)
print('✅ Saved N-body_Simulation_&_Halo_Formation.mp4')


```
---

## ⚙️ <a name="Code_for_N-body_Simulation_axes"></a>3. Code for N-body Simulation & Halo Formation (with axes)

```python
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.spatial import cKDTree

#Extract Redshift Value
def extract_redshift1(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1] + "." + parts[2])

def extract_z_nbody(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1] + "." + parts[2])

def extract_z_halo(fn):
    base = os.path.splitext(os.path.basename(fn))[0]
    parts = base.split("_")
    return float(parts[1])

#Load (N-body only)
files1 = sorted(glob.glob("p_*.txt"), key=extract_redshift1, reverse=True)
z1 = [extract_redshift1(f) for f in files1]
raw1 = [np.loadtxt(f).reshape(-1, 3) for f in files1]

frames1, zs1 = [], []
for (a, za), (b, zb) in zip(zip(raw1[:-1], z1[:-1]), zip(raw1[1:], z1[1:])):
    frames1.append(a); zs1.append(za)
    frames1.append((a + b) / 2); zs1.append((za + zb) / 2)
frames1.append(raw1[-1]); zs1.append(z1[-1])

#Load (N-body for center panel)
frames2_n = frames1.copy()

#I have assumed that the N-body Data Files are in the same directory as this notebook
#The Halo Files are in a different directory

#If n-body data files are also in a different directory, the same approach as below can be taken for n-body as well

#Load (halo-only) from external path
halo_dir = "/Users/pragunnepal/Downloads/500Halos"
halo_files = sorted(glob.glob(os.path.join(halo_dir, "halo_*.txt")), key=extract_z_halo, reverse=True)
z_h = [extract_z_halo(f) for f in halo_files]
raw_h = [np.loadtxt(f).reshape(-1, 4) for f in halo_files]

#Interpolate halo data for right and center (same frames3)

frames3 = []
for z in zs1:
    if z in z_h:
        d3 = raw_h[z_h.index(z)]
    else:
        all_z = np.array(z_h)
        if len(all_z) == 0:
            d3 = np.zeros((0, 4))
        elif z <= all_z[-1]:
            d3 = raw_h[-1]
        elif z >= all_z[0]:
            d3 = raw_h[0]
        else:
            i = np.searchsorted(-all_z, -z)
            i = np.clip(i, 1, len(all_z) - 1)
            zA, zB = z_h[i - 1], z_h[i]
            A, B = raw_h[i - 1], raw_h[i]
            t = (z - zA) / (zB - zA) if zB != zA else 0
            if A.size == 0 or B.size == 0:
                d3 = A if B.size == 0 else B
            else:
                tree = cKDTree(A[:, 1:])
                _, idxs = tree.query(B[:, 1:], k=1)
                A_matched = A[idxs]
                d3 = (1 - t) * A_matched + t * B
    frames3.append(d3)

frames2 = list(zip(frames2_n, frames3, zs1))

#Animation setup

fps = 30
extra_hold = fps * 33
zoom_in, zoom_out, max_zoom = fps * 15, fps * 5, 1.2
motion = len(frames1)
total_frames = motion + extra_hold

samples = np.vstack([fr[:, :3] for fr in frames1] + [fr[0] for fr in frames2] + [fr[:, 1:] for fr in frames3 if fr.size > 0])
bounds = [(samples[:, i].min(), samples[:, i].max()) for i in range(3)]
bounds = [(mn - 0.2 * (mx - mn), mx + 0.2 * (mx - mn)) for mn, mx in bounds]
centers = [0.5 * (mn + mx) for mn, mx in bounds]

dpi = 300
fig = plt.figure(figsize=(18, 6), dpi=dpi)
axes = [fig.add_subplot(1, 3, i + 1, projection='3d') for i in range(3)]

for ax in axes:
    ax.set_facecolor('black')
    fig.patch.set_facecolor('black')
    ax.set_box_aspect([1, 1, 1])
    ax.axis('on')
    for axis in [ax.xaxis, ax.yaxis, ax.zaxis]:
        axis._axinfo['grid']['color'] = 'purple'
        axis._axinfo['grid']['linewidth'] = 0.1
        axis._axinfo['grid']['alpha'] = 0.001  # almost invisible
        axis.pane.set_facecolor('purple')
        axis.pane.set_edgecolor('purple')
        axis.pane.set_alpha(0.001)  # almost invisible
    ax.tick_params(colors='purple', which='both', labelsize=6)

txt = axes[0].text2D(0.02, 0.95, '', transform=axes[0].transAxes, color='white')
ax_triad = inset_axes(axes[0], width="20%", height="20%", loc='lower left')
ax_triad.axis('off')

scats1 = axes[0].scatter([], [], [], s=0.5, alpha=0.1, c='cyan', edgecolors='none')
scats2_n = axes[1].scatter([], [], [], s=0.5, alpha=0.1, c='cyan', edgecolors='none')
scats2_h = axes[1].scatter([], [], [], s=[], alpha=0.7, c='gold', edgecolors='none')
scats3 = axes[2].scatter([], [], [], s=[], c='gold', alpha=0.7, edgecolors='none')

def rotation_matrix(angle_deg):
    theta = np.deg2rad(angle_deg)
    Rz = np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
    Rx = np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
    return Rz @ Rx

def update(frame):
    idx = frame if frame < motion else motion - 1
    d1 = frames1[idx]
    scats1._offsets3d = (d1[:, 0], d1[:, 1], d1[:, 2])
    dn, dh, _ = frames2[idx]
    scats2_n._offsets3d = (dn[:, 0], dn[:, 1], dn[:, 2])
    if dh.size:
        scats2_h._offsets3d = (dh[:, 1], dh[:, 2], dh[:, 3])
        scats2_h.set_sizes(np.cbrt(np.ones(len(dh))) * 30)
    else:
        scats2_h._offsets3d = ([], [], [])
        scats2_h.set_sizes([])
    d3 = frames3[idx]
    if d3.size:
        coords3 = d3[:, 1:]
        sizes3 = (np.cbrt(d3[:, 0]) / np.cbrt(d3[:, 0]).max()) * 50
        scats3._offsets3d = (coords3[:, 0], coords3[:, 1], coords3[:, 2])
        scats3.set_sizes(sizes3)
    else:
        scats3._offsets3d = ([], [], [])
        scats3.set_sizes([])
    txt.set_text(f'z = {zs1[idx]:.3f}')
    angle = frame * 0.1
    for ax in axes:
        ax.view_init(25 + angle, 30 + angle)
    ax_triad.clear()
    ax_triad.axis('off')
    R = rotation_matrix(angle)
    for color, vec, label in zip(['red', 'green', 'blue'], [np.array([1, 0, 0]), np.array([0, 1, 0]), np.array([0, 0, 1])], ['X', 'Y', 'Z']):
        vrot = R @ vec
        end = np.array([0.5, 0.5]) + 0.3 * vrot[:2]
        ax_triad.annotate('', xy=end, xytext=(0.5, 0.5), arrowprops=dict(arrowstyle='->', color=color))
        ax_triad.text(end[0], end[1], label, color=color, fontsize=8)
    if frame >= motion:
        h = frame - motion
        if h <= zoom_in:
            f = 1 + (max_zoom - 1) * (h / zoom_in)
        else:
            f = max_zoom - (max_zoom - 1) * min((h - zoom_in) / zoom_out, 1)
    else:
        f = 1
    for i, (mn, mx) in enumerate(bounds):
        c = (mn + mx) / 2
        r = (mx - mn) / 2
        low, high = c - r / f, c + r / f
        for ax in axes:
            if i == 0:
                ax.set_xlim(low, high)
            elif i == 1:
                ax.set_ylim(low, high)
            else:
                ax.set_zlim(low, high)
    return scats1, scats2_n, scats2_h, scats3, txt

ani = FuncAnimation(fig, update, frames=total_frames, interval=1000 / fps, blit=False)
writer = FFMpegWriter(fps=fps, metadata={'PragunNepal': 'Combined'}, bitrate=30000)
ani.save('N-body_Simulation_&_Halo_Formation_with_axes.mp4', writer=writer)
print('✅ Saved N-body_Simulation_&_Halo_Formation_with_axes.mp4')
```
---

## 🔭 <a name="conclusion"></a>4. Conclusion

The code has been made general, hence its easily customisable. Please make necessary changes to the code as per your needs. 

©Pragun Nepal, 2025

Contact Me :
- Email : pragunnepal@gmail.com
- Email : pragun23@iisertvm.ac.in
- GitHub : PragunNepal
- LinkedIn : Pragun Nepal